In [128]:
import requests
import pandas as pd
import numpy as np

In [357]:
endpoint = "https://www.climatewatchdata.org/api/v1/data/historical_emissions"
parameters = {'regions': ['BRA','MEX'],
              'start_year': 1990,
              'end_year': 2018,
              'sort_col': 'year',
              'sort_dir': 'DESC'}
historical_emissions = requests.get(endpoint, params = parameters)

In [358]:
#Verificamos que salio bien nuestro requests
historical_emissions.status_code

200

In [359]:
json = historical_emissions.json()
json.keys()

dict_keys(['data', 'meta'])

In [360]:
data = json['data']
normalized = pd.json_normalize(data)
df = pd.DataFrame.from_dict(normalized)
df.head()

,id,iso_code3,country,data_source,sector,gas,unit,emissions
0,658228,MEX,Mexico,CAIT,Total excluding LUCF,All GHG,MtCO₂e,"[{'year': 2011, 'value': 664.66}, {'year': 199..."
1,658229,MEX,Mexico,CAIT,Total including LUCF,All GHG,MtCO₂e,"[{'year': 2003, 'value': 492.69}, {'year': 201..."
2,658230,MEX,Mexico,CAIT,Energy,All GHG,MtCO₂e,"[{'year': 2017, 'value': 495.63}, {'year': 199..."
3,658231,MEX,Mexico,CAIT,Industrial Processes,All GHG,MtCO₂e,"[{'year': 2003, 'value': 21.35}, {'year': 1990..."
4,658232,MEX,Mexico,CAIT,Agriculture,All GHG,MtCO₂e,"[{'year': 1997, 'value': 85.51}, {'year': 1998..."


# Aquí hay un problema solo está detectando un país

In [361]:
df['country'].unique()

array(['Mexico'], dtype=object)

Observemos la columna de emisiones la cual no pudo ser normalizada, sin emabargo está claro que es una lista de diccionarios, por lo que vamos a extraer estos datos por separado.

In [317]:
# Observemos los elementos del diccionario dos llaves('year' y 'value') y sus respectivos valores
print(df['emissions'][0][0].items())
print(df['emissions'][0][0].keys())
df['emissions'][0][0].values()

dict_items([('year', 1990), ('value', 593.95)])
dict_keys(['year', 'value'])


dict_values([1990, 593.95])

In [318]:
#Con get() podemos acceder al valor de cada llave del diccionario
print(df['emissions'][0][0].get('value'))
df['emissions'][0][0].get('year')

593.95


1990

In [319]:
# Definimos una funcion para extraer los datos

def extraccion(lista, pais):

  # Definimos listas donde almacenaremos los datos extraidos (años, valor, pais)
  lista_1 =[]
  lista_2 = []
  paises = []

  for i in range(len(lista)):
    for y in range(len(lista[i])):

      # Extraemos el pais
      paises.append(pais[i])
 
      # Extraemos los años
      anio = lista[i][y].get('year')
      lista_1.append(anio)

      # Extraemos los valores
      valor = lista[i][y].get('value')
      lista_2.append(valor)

  # Unimos en un df
  data = pd.DataFrame({'country': paises, 'year': lista_1, 'value': lista_2})

  return data

In [320]:
data = extraccion(df["emissions"], df["country"])
data.head()

,country,year,value
0,Brazil,1990,593.95
1,Brazil,1991,621.22
2,Brazil,1992,631.15
3,Brazil,1993,641.36
4,Brazil,1994,657.69
